In [72]:
import numpy as np                  # for basic operations over arrays
from scipy.spatial import distance  # to compute the Gaussian kernel
import cvxopt                       # to solve the dual optimization problem
import copy 
from sklearn.datasets import load_iris, load_breast_cancer, load_digits, fetch_openml
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
#KERNELS

kernel_funs = {
    'linear': lambda x, xࠤ , c=0: x @ xࠤ .T, 
    'polynomial': lambda x, xࠤ , Q=5: (1 + x @ xࠤ.T)**Q, 
    'rbf': lambda x, xࠤ , γ=10: np.exp(-γ * distance.cdist(x, xࠤ,'sqeuclidean')), 
    'sigmoid': lambda x, xࠤ , γ=10, r=0: np.tanh(γ * x @ xࠤ.T + r)
    }

In [6]:
class SVM:
    def __init__(self, kernel: str='linear', C: int=1, k: int=2):
        self.kernel_str = kernel
        self.kernel = kernel_funs[kernel] 
        self.C = C                  
        self.k = k                 
        self.X, y = None, None
        self.αs = None
        self.multiclass = False
        self.clfs = []    
    
    def fit(self, X, y, eval_train=False):
        if len(np.unique(y)) > 2:
            self.multiclass = True
            return self.multi_fit(X, y, eval_train)
        
        # relabel if needed
        if set(np.unique(y)) == {0, 1}: y[y == 0] = -1
        # ensure y has dimensions Nx1
        self.y = y.reshape(-1, 1).astype(np.double) # Has to be a column vector
        self.X = X
        N = X.shape[0]
        
        # compute the kernel over all possible pairs of (x, x') in the data
        self.K = self.kernel(X, X, self.k)
        
        # For 1/2 x^T P x + q^T x
        P = cvxopt.matrix(self.y @ self.y.T * self.K)
        q = cvxopt.matrix(-np.ones((N, 1)))
        
        # For Ax = b
        A = cvxopt.matrix(self.y.T)
        b = cvxopt.matrix(np.zeros(1))

        # For Gx <= h
        G = cvxopt.matrix(np.vstack((-np.identity(N),
                                    np.identity(N))))
        h = cvxopt.matrix(np.vstack((np.zeros((N,1)),
                                    np.ones((N,1)) * self.C)))

        # Solve    
        cvxopt.solvers.options['show_progress'] = False
        sol = cvxopt.solvers.qp(P, q, G, h, A, b)
        self.αs = np.array(sol["x"])
            
        # Maps into support vectors
        self.is_sv = ((self.αs > 1e-3) & (self.αs <= self.C)).squeeze()
        self.margin_sv = np.argmax((1e-3 < self.αs) & (self.αs < self.C - 1e-3))
        
        if eval_train:  
            print(f"Finished training with accuracy {self.evaluate(X, y)}")
    
    def multi_fit(self, X, y, eval_train=False):
        self.k = len(np.unique(y))      # number of classes
        # for each pair of classes
        for i in range(self.k):
            # get the data for the pair
            Xs, Ys = X, copy.copy(y)
            # change the labels to -1 and 1
            Ys[Ys!=i], Ys[Ys==i] = -1, +1
            # fit the classifier
            clf = SVM(kernel=self.kernel_str, C=self.C, k=self.k)
            clf.fit(Xs, Ys)
            # save the classifier
            self.clfs.append(clf)
        if eval_train:  
            print(f"Finished training with accuracy {self.evaluate(X, y)}")

    def predict(self, X_t):
        if self.multiclass: return self.multi_predict(X_t)
        xₛ, yₛ = self.X[self.margin_sv, np.newaxis], self.y[self.margin_sv]
        αs, y, X= self.αs[self.is_sv], self.y[self.is_sv], self.X[self.is_sv]

        b = yₛ - np.sum(αs * y * self.kernel(X, xₛ, self.k), axis=0)
        score = np.sum(αs * y * self.kernel(X, X_t, self.k), axis=0) + b
        return np.sign(score).astype(int), score 
    
    def multi_predict(self, X):
        preds = np.zeros((X.shape[0], self.k))
        for i, clf in enumerate(self.clfs):
            _, preds[:, i] = clf.predict(X)
     
        return np.argmax(preds, axis=1)
    
    def evaluate(self, X,y):
        if self.multiclass: 
            outputs = self.multi_predict(X)
        else:
            outputs, _ = self.predict(X)
            
        accuracy = np.sum(outputs == y) / len(y)
        return round(accuracy, 2)

In [87]:
#SVM for Iris dataset

X, y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

for kernel in kernel_funs.keys():
    print(f"Kernel: {kernel}")
    svm = SVM(kernel=kernel)
    svm.fit(X_train, y_train, eval_train=True)

    y_pred = svm.predict(X_test)
    print(f"Test accuracy for {kernel} kernel: {np.sum(y_test==y_pred)/y_test.shape[0]:.2f}\n")


Kernel: linear
Finished training with accuracy 0.97
Test accuracy for linear kernel: 0.87

Kernel: polynomial
Finished training with accuracy 0.62
Test accuracy for polynomial kernel: 0.63

Kernel: rbf
Finished training with accuracy 0.98
Test accuracy for rbf kernel: 0.97

Kernel: sigmoid
Finished training with accuracy 0.37
Test accuracy for sigmoid kernel: 0.20



In [88]:
X, y = fetch_openml('titanic', version=1, return_X_y=True)
y = y.map({'0': 0, '1': 1}).astype(int).to_numpy()
X = X[['age', 'sex', 'pclass']]
X['sex'] = X['sex'].map({'male': 0, 'female': 1}).astype(int)
X = X.fillna(X.mean()).to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

for kernel in kernel_funs.keys():
    print(f"Kernel: {kernel}")
    svm = SVM(kernel=kernel)
    svm.fit(X_train, y_train, eval_train=True)

    y_pred = svm.predict(X_test)
    print(f"Test accuracy for {kernel} kernel: {np.sum(y_test==y_pred)/y_test.shape[0]:.2f}\n")


Kernel: linear
Finished training with accuracy 0.77
Test accuracy for linear kernel: 0.29

Kernel: polynomial
Finished training with accuracy 0.71
Test accuracy for polynomial kernel: 0.26

Kernel: rbf
Finished training with accuracy 0.84
Test accuracy for rbf kernel: 0.23

Kernel: sigmoid
Finished training with accuracy 0.62
Test accuracy for sigmoid kernel: 0.00



In [89]:
#SVM for Breast Cancer dataset

X, y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

for kernel in kernel_funs.keys():
    print(f"Kernel: {kernel}")
    svm = SVM(kernel=kernel)
    svm.fit(X_train, y_train, eval_train=True)

    y_pred = svm.predict(X_test)
    print(f"Test accuracy for {kernel} kernel: {np.sum(y_test==y_pred)/y_test.shape[0]:.2f}\n")

Kernel: linear
Finished training with accuracy 0.97
Test accuracy for linear kernel: 0.63

Kernel: polynomial
Finished training with accuracy 0.37
Test accuracy for polynomial kernel: 0.00

Kernel: rbf
Finished training with accuracy 1.0
Test accuracy for rbf kernel: 0.63

Kernel: sigmoid
Finished training with accuracy 0.63
Test accuracy for sigmoid kernel: 0.63



In [90]:
#SVM for Digits dataset

X, y = load_digits(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

for kernel in kernel_funs.keys():
    print(f"Kernel: {kernel}")
    svm = SVM(kernel=kernel)
    svm.fit(X_train, y_train, eval_train=True)

    y_pred = svm.predict(X_test)
    print(f"Test accuracy for {kernel} kernel: {np.sum(y_test==y_pred)/y_test.shape[0]:.2f}\n")

Kernel: linear
Finished training with accuracy 0.98
Test accuracy for linear kernel: 0.94

Kernel: polynomial
Finished training with accuracy 0.1
Test accuracy for polynomial kernel: 0.11

Kernel: rbf
Finished training with accuracy 1.0
Test accuracy for rbf kernel: 0.08

Kernel: sigmoid
Finished training with accuracy 0.1
Test accuracy for sigmoid kernel: 0.09

